# Tiingo

In [1]:
import pandas as pd
import requests
import datetime

In [2]:
# lecture du fichier spot binance
spot = pd.read_csv(r"C:\Users\open_\Downloads\Binance_spot.csv")
display(spot.head())

print(f"Nombre de tickers : {spot.shape[0]}")

,web-scraper-order,web-scraper-start-url,Pagination,Ticker
0,1693047515-1,https://www.binance.com/fr/markets/spot_margin...,NaN,BTC
1,1693047515-2,https://www.binance.com/fr/markets/spot_margin...,NaN,ETH
2,1693047515-3,https://www.binance.com/fr/markets/spot_margin...,NaN,USDC
3,1693047515-4,https://www.binance.com/fr/markets/spot_margin...,NaN,XRP
4,1693047515-5,https://www.binance.com/fr/markets/spot_margin...,NaN,TUSD


Nombre de tickers : 349


In [92]:
# Liste en ordre croissant
spot_list = spot["Ticker"].sort_values()

# en Minuscule
lower_spot_list = [element.lower() for element in spot_list]

# Ajout usd
ticker_usd = [element + "usd" for element in lower_spot_list]

print(f"APERCU LISTE TICKERS : {ticker_usd[:5]} ...")
print(f"Nombre de tickers : {len(ticker_usd)}")

APERCU LISTE TICKERS : ['1inchusd', 'aaveusd', 'acausd', 'achusd', 'acmusd'] ...
Nombre de tickers : 349


In [5]:
# Tickers + prices
data_list = []

for t in ticker_usd:
    today = datetime.date.today()
    headers = {'Content-Type': 'application/json'}
    requestResponse = requests.get(f"https://api.tiingo.com/tiingo/crypto/prices?tickers={t}&startDate={today}&resampleFreq=1Day&token=bd85eb065e5b5c76d82c98f43f036851c7bbef4d", headers=headers)

    data = requestResponse.json()
    for item in data:
        ticker = item["ticker"]
        price = item["priceData"]
        
        for entry in price:
            close = entry["close"]
            data_list.append({'Ticker': ticker, 'Close': close})
            
# DataFrame à partir de la liste Tickers + prices
df = pd.DataFrame(data_list)

print(df)

# Yahoo Finance

In [66]:
# Liste en ordre croissant
spot_list = spot["Ticker"].sort_values()

# en Majuscule
upper_spot_list = [element.upper() for element in spot_list]

# Ajout "-USD"
ticker_usd_up = [element + "-USD" for element in upper_spot_list]

print(f"APERCU LISTE TICKERS : {ticker_usd_up[:5]} ...")
print(f"Nombre de tickers : {len(ticker_usd_up)}")

APERCU LISTE TICKERS : ['1INCH-USD', 'AAVE-USD', 'ACA-USD', 'ACH-USD', 'ACM-USD'] ...
Nombre de tickers : 349


In [47]:
import yfinance as yf

# Liste des symboles des instruments financiers
symbols = ticker_usd_up

# Période temporelle de début et de fin
start_date = '2023-08-26'
end_date = '2023-08-27'

# Récupération des données sous forme de DataFrame
data_yahoo = yf.download(symbols, start=start_date, end=end_date)

[*********************100%***********************]  349 of 349 completed

32 Failed downloads:
- COMP-USD: No data found for this date range, symbol may be delisted
- SYN-USD: No data found for this date range, symbol may be delisted
- MAGIC-USD: No data found for this date range, symbol may be delisted
- GRT-USD: No data found for this date range, symbol may be delisted
- PLA-USD: No data found for this date range, symbol may be delisted
- GBP-USD: No timezone found, symbol may be delisted
- ACM-USD: No data found for this date range, symbol may be delisted
- MDX-USD: No data found for this date range, symbol may be delisted
- STG-USD: Period 'max' is invalid, must be one of ['1d', '5d']
- CITY-USD: No data found for this date range, symbol may be delisted
- PNT-USD: No data found for this date range, symbol may be delisted
- PROS-USD: No data found for this date range, symbol may be delisted
- REI-USD: No data found for this date range, symbol may be delisted
- EDU-USD: No data found

In [73]:
# Affichage du DataFrame
display(data_yahoo)

Adj Close                                         \
                    1INCH-USD   AAVE-USD   ACA-USD   ACH-USD ACM-USD   
Date                                                                   
2023-08-26 00:00:00  0.243045  56.641541  0.045165  0.013644     NaN   

                                                                      ...  \
                      ADA-USD  ADX-USD AERGO-USD  AGIX-USD  AGLD-USD  ...   
Date                                                                  ...   
2023-08-26 00:00:00  0.260953  0.12727  0.095344  0.180171  0.489546  ...   

                       Volume                                         \
                      XNO-USD    XRP-USD   XTZ-USD  XVG-USD  XVS-USD   
Date                                                                   
2023-08-26 00:00:00  10191011  541153725  31653800  2624893  1075514   

                                                                     
                      YFI-USD   YGG-USD   ZEC-USD  ZIL-USD  ZRX-USD  
Date                                                                 
2023-08-26 00:00:00  20905746  28944059  38775557  7404658  5181172  

[1 rows x 2094 columns]

In [93]:
# Mise à plat du DataFrame avec la méthode stack
flattened_df = data_yahoo.stack()

# Réinitialisation de l'index
flattened_df = flattened_df.reset_index()
display(flattened_df.head())

# Données manquantes
len_init_data = len(ticker_usd_up)
len_dl_data = flattened_df.shape[0]
miss_data = len_init_data - len_dl_data
print(f"Données en requête : {len_init_data}\nDonnées Yahoo Finance : {len_dl_data}\nDonnées manquantes : {miss_data}")

,Date,level_1,Adj Close,Close,High,Low,Open,Volume
0,2023-08-26,1INCH-USD,0.243045,0.243045,0.249031,0.242388,0.246625,13316640.0
1,2023-08-26,AAVE-USD,56.641541,56.641541,57.273891,56.322121,56.884418,51956085.0
2,2023-08-26,ACA-USD,0.045165,0.045165,0.046065,0.045026,0.045557,2900438.0
3,2023-08-26,ACH-USD,0.013644,0.013644,0.013867,0.013504,0.013789,3217837.0
4,2023-08-26,ADA-USD,0.260953,0.260953,0.261614,0.258455,0.261075,131669884.0


Données en requête : 349
Données Yahoo Finance : 317
Données manquantes : 32


In [89]:
# Récupération des colonnes nécessaires
binance_spot = flattened_df[["level_1", "Adj Close"]]
binance_spot = binance_spot.copy()
binance_spot.rename(columns={"level_1":"Instrument", "Adj Close":"Clôture"}, inplace=True)

# Affichage du DataFrame
display(binance_spot.head())

,Instrument,Clôture
0,1INCH-USD,0.243045
1,AAVE-USD,56.641541
2,ACA-USD,0.045165
3,ACH-USD,0.013644
4,ADA-USD,0.260953


In [90]:
binance_spot.to_csv(r"C:\Users\open_\Downloads\données_financières.csv")